# Cookbook: Drive Guide

In this cookbook, we will demonstrate how to query the distance between two locations and create a driving plan using an agent.

# Install Requirements

We will use [Qwen-Agent](https://github.com/QwenLM/Qwen-Agent) in this book. For demonstration, we will need at least the `mcp` functionality of Qwen-Agent.

In [ ]:
!pip3 install -U "qwen-agent[gui,rag,code_interpreter,mcp]"
# `pip install -U qwen-agent` will install the minimal requirements.
# The optional requirements, specified in double brackets, are:
#   [gui] for Gradio-based GUI support;
#   [rag] for RAG support;
#   [code_interpreter] for Code Interpreter support;
#   [mcp] for MCP support.

# Step 1: Create an Agent

Create an agent that is capable of utilizing map functionalities provided by Amap.

An api key for Amap is needed, instructions are at https://lbs.amap.com/api/mcp-server/summary

In [1]:
from qwen_agent.agents import Assistant
from qwen_agent.utils.output_beautify import typewriter_print
# `typewriter_print` prints streaming messages in a non-overlapping manner for a clear view.

In [2]:
llm_cfg = {
    'model': 'qwen3-32b',
    'model_server': 'dashscope',
    'api_key': '' # **fill your dashscope api key here**

    # Use a model service compatible with the OpenAI API, such as vLLM or Ollama:
    # 'model': 'Qwen3-8B',
    # 'model_server': 'http://localhost:8000/v1',  # base_url, also known as api_base
    # 'api_key': 'EMPTY'
}

tools = [
    {
        "mcpServers": {
            # enumeration of mcp server configs
            "amap-amap-sse": {
                "url": "https://mcp.amap.com/sse?key=YOUR_KEY" # **fill your amap mcp key**
            }
        }
    }
]

agent = Assistant(
    llm=llm_cfg, 
    function_list=tools
)

2025-05-01 21:12:51,768 - mcp_manager.py - 110 - INFO - Initializing MCP tools from mcp servers: ['amap-amap-sse']


# Step 2: Calculate the Distance

We can query the distance between two locations by prompting the agent, which will then provide an answer utilizing the MCP server.

In [3]:
query = "How far is it between Zhejiang University (more specifically, the Zijingang Campus) and Hangzhou Xiaoshan airport?"
messages = [{'role': 'user', 'content': query}]

In [4]:
response_plain_text = ''

for ret_messages in agent.run(messages):
    response_plain_text = typewriter_print(ret_messages, response_plain_text)

[THINK]
Okay, the user is asking for the distance between Zhejiang University's Zijingang Campus and Hangzhou Xiaoshan Airport. Let me think about how to approach this.

First, I need to get the coordinates for both locations. For Zijingang Campus, I can use the amap-amap-sse-maps_geo function to convert the address into coordinates. The address would be something like "Zijingang Campus, Zhejiang University, Hangzhou, China". Similarly, for Hangzhou Xiaoshan Airport, the address is "Hangzhou Xiaoshan International Airport, Hangzhou, China".

Once I have both sets of coordinates, I can use the amap-amap-sse-maps_distance function to calculate the distance. The type parameter should be set to 1 for driving distance, as the user probably wants the driving distance rather than straight-line or walking.

Wait, but maybe I should check if there's a direct function for distance between two addresses. But looking at the available tools, the maps_distance function requires coordinates. So first

In [5]:
messages += ret_messages # extending the context with new `ret_messages`.

# Step 3: Make a Drive Plan

We can create a drive plan by asking follow-up questions, and the agent will generate the plan for us.

In [6]:
messages += [{'role': 'user', 'content': 'Okay, how can i drive?'}]

In [7]:
response_plain_text = ''

for ret_messages in agent.run(messages):
    response_plain_text = typewriter_print(ret_messages, response_plain_text)

[THINK]
Okay, the user asked for driving directions from Zhejiang University's Zijingang Campus to Hangzhou Xiaoshan Airport. I need to figure out which function to use.

First, I remember that there's a function for driving route planning. Let me check the tools provided. The function is called "amap-amap-sse-maps_direction_driving". It requires origin and destination coordinates. 

Earlier, I used the "amap-amap-sse-maps_geo" function to get the coordinates for both locations. The Zijingang Campus is at 120.088667,30.304176 and the airport is at 120.427708,30.228153. So those coordinates should be used as the origin and destination parameters.

I need to make sure the parameters are correctly formatted. The origin and destination should be strings in "经度,纬度" format. The function doesn't require any other parameters, so just those two should be enough. 

Let me structure the tool call with the correct function name and arguments. Double-checking the required fields, both origin and de

# Use the GUI

We have explored the capabilities of the Qwen-Agent framework and Qwen models in providing travel suggestions. We can also achieve this by using the GUI.

In [8]:
from qwen_agent.gui import WebUI

agent = Assistant(
    name="Qwen Guide",
    description="I can help with your travel needs, ask me anything!",
    llm=llm_cfg,
    function_list=tools
)

WebUI(agent).run()

2025-05-01 21:14:00,386 - mcp_manager.py - 110 - INFO - Initializing MCP tools from mcp servers: ['amap-amap-sse']


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
